In [1]:
import warnings
warnings.filterwarnings("ignore")

from util import get_num_lines, get_pos2idx_idx2pos, index_sequence, get_vocab, embed_indexed_sequence, \
    get_word2idx_idx2word, get_embedding_matrix, write_predictions, get_performance_VUAverb_val
from util import TextDatasetWithGloveElmoSuffix as TextDataset
from util import evaluate
from model import RNNSequenceModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

from tqdm import tqdm
import csv
import h5py
import random
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print("PyTorch version:")
print(torch.__version__)
#print("GPU Detected:")
#print(torch.cuda.is_available())
using_GPU = False

PyTorch version:
0.4.1


## 1. Data pre-processing

Читаем датасет как list, каждый элемент содержит 3:
   * предложение: строка
   * индекс рассматриваемого слова
   * лэйбл: 1 или 0

In [6]:
# для датасета "атмосфера"
raw_mohx = []
new_ones, new_zeros = 0, 0
ones = 0 # количество лэйблов 1
zeros = 0 # количество лэйблов 0

with open('../data/атмосфера.csv', encoding='CP1251') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        sentence = line[3]
        label_seq = [0] * len(sentence.split())
        pos_seq = [0] * len(label_seq)
        verb_idx = int(line[4])
        verb_label = int(line[5])
        if verb_label == 1:
            ones += 1
            label_seq[verb_idx] = verb_label
            pos_seq[verb_idx] = 1   # idx2pos = {0: 'words that are not focus verbs', 1: 'focus verb'}
            raw_mohx.append([sentence.strip(), label_seq, pos_seq])
            #raw_mohx.append([sentence.strip(), label_seq, pos_seq])
        else:
            zeros += 1
            if zeros < 200000:
                label_seq[verb_idx] = verb_label
                new_zeros += 1
                pos_seq[verb_idx] = 1   # idx2pos = {0: 'words that are not focus verbs', 1: 'focus verb'}
                raw_mohx.append([sentence.strip(), label_seq, pos_seq])
                #raw_mohx.append([sentence.strip(), label_seq, pos_seq])

print('Размер датасета: ', len(raw_mohx))
print('Примеров с лэйблом 1: {}%'.format(ones/(ones+new_zeros)*100))

Размер датасета:  2436
Примеров с лэйблом 1: 48.275862068965516%


In [3]:
# для собранных со всех датасетов, полученных из института русского языка
# соотношение классов сделано примерно 50:50
raw_mohx = []
new_ones, new_zeros = 0, 0
ones = 0 # количество лэйблов 1
zeros = 0 # количество лэйблов 0

datasets = ['атмосфера','ад','агония','ангел','алтарь','архитектор','акула','атом','артерия','алмаз']

#with open('../data/MOH-X/MOH-X_formatted_svo_cleaned.csv') as f:
#with open('../data/master_marga.csv') as f:
for name in datasets:
    with open('../data/' + name + '.csv', encoding='CP1251') as f:
    # arg1  	arg2	verb	sentence	verb_idx	label
        lines = csv.reader(f)
        next(lines)
        for line in lines:
            sentence = line[3]
            label_seq = [0] * len(sentence.split())
            pos_seq = [0] * len(label_seq)
            verb_idx = int(line[4])
            verb_label = int(line[5])
            if verb_label == 1:
                ones += 1
                label_seq[verb_idx] = verb_label
                pos_seq[verb_idx] = 1   # idx2pos = {0: 'words that are not focus verbs', 1: 'focus verb'}
                raw_mohx.append([sentence.strip(), label_seq, pos_seq])
                #raw_mohx.append([sentence.strip(), label_seq, pos_seq])
            else:
                zeros += 1
                if zeros < 5277:
                    label_seq[verb_idx] = verb_label
                    new_zeros += 1
                    pos_seq[verb_idx] = 1   # idx2pos = {0: 'words that are not focus verbs', 1: 'focus verb'}
                    raw_mohx.append([sentence.strip(), label_seq, pos_seq])
                    #raw_mohx.append([sentence.strip(), label_seq, pos_seq])

print('Размер датасета: ', len(raw_mohx))
print('Примеров с лэйблом 1: {}%'.format(ones/(ones+new_zeros)*100))

Размер датасета:  10000
Примеров с лэйблом 1: 47.24%


In [4]:
print('Примеров с лэйблом 1 суммарно без обрезания датасета: {}%'.format(ones/(ones+zeros)*100))

Примеров с лэйблом 1 суммарно без обрезания датасета: 23.129651390520955%


## 2. Data preparation

Получаем словарь и эмбеддинги (использовался glove для английского и https://rusvectores.org/ru/about/ для русского)

In [5]:
# vocab is a set of words
vocab = get_vocab(raw_mohx)
# two dictionaries. <PAD>: 0, <UNK>: 1
word2idx, idx2word = get_word2idx_idx2word(vocab)
# glove_embeddings a nn.Embeddings
glove_embeddings = get_embedding_matrix(word2idx, idx2word, normalization=False)
# elmo_embeddings
# set elmos_mohx=None to exclude elmo vectors. Also need to change the embedding_dim in later model initialization
#elmos_mohx = h5py.File('../elmo/MOH-X_cleaned.hdf5', 'r')
elmos_mohx=None

random.seed(0)
random.shuffle(raw_mohx)

# second argument is the post sequence, which we don't need
embedded_mohx = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_mohx, None),
                       example[2], example[1]]
                      for example in raw_mohx]

vocab size:  96613


100%|██████████| 189194/189194 [00:10<00:00, 18071.89it/s]


Number of pre-trained word vectors loaded:  16690
Embeddings mean:  0.0073782335966825485
Embeddings stdev:  0.7526317834854126


10-фолдная кросс-валидация:

In [9]:
# separate the embedded_sentences and labels into 2 list, in order to pass into the TextDataset as argument
sentences = [example[0] for example in embedded_mohx]
poss = [example[1] for example in embedded_mohx]
labels = [example[2] for example in embedded_mohx]
# ten_folds is a list of 10 tuples, each tuple is (list_of_embedded_sentences, list_of_corresponding_labels)
ten_folds = []
fold_size = int(10000 / 10)
for i in range(10):
    ten_folds.append((sentences[i * fold_size:(i + 1) * fold_size],
                      poss[i * fold_size:(i + 1) * fold_size],
                      labels[i * fold_size:(i + 1) * fold_size]))

idx2pos = {0: 'words that are not focus verbs', 1: 'focus verb'}

optimal_f1s = []
optimal_ps = []
optimal_rs = []
optimal_accs = []
predictions_all = []
for i in range(10):
    '''
    2. 3
    set up Dataloader for batching
    '''
    training_sentences = []
    training_labels = []
    training_poss = []
    for j in range(10):
        if j != i:
            training_sentences.extend(ten_folds[j][0])
            training_poss.extend(ten_folds[j][1])
            training_labels.extend(ten_folds[j][2])
    training_dataset_mohx = TextDataset(training_sentences, training_poss, training_labels)
    val_dataset_mohx = TextDataset(ten_folds[i][0], ten_folds[i][1], ten_folds[i][2])
    #print(len(val_dataset_mohx))
    # Data-related hyperparameters
    batch_size = 10
    # Set up a DataLoader for the training, validation, and test dataset
    train_dataloader_mohx = DataLoader(dataset=training_dataset_mohx, batch_size=batch_size, shuffle=True,
                                       collate_fn=TextDataset.collate_fn)
    val_dataloader_mohx = DataLoader(dataset=val_dataset_mohx, batch_size=batch_size, shuffle=False,
                                     collate_fn=TextDataset.collate_fn)


## 3. Model training

Настройка модели, loss и оптимизатора

In [10]:

    # Instantiate the model
    # embedding_dim = glove + elmo + suffix indicator
    # dropout1: dropout on input to RNN
    # dropout2: dropout in RNN; would be used if num_layers=1
    # dropout3: dropout on hidden state of RNN to linear layer
    
    # with elmo embedding_dim=300+1024
RNNseq_model = RNNSequenceModel(num_classes=2, embedding_dim=300, hidden_size=300, 
                                    num_layers=1, bidir=True,
                                    dropout1=0.5, dropout2=0, dropout3=0)
    # Move the model to the GPU if available
if using_GPU:
    RNNseq_model = RNNseq_model.cuda()
    # Set up criterion for calculating loss
loss_criterion = nn.NLLLoss()
    # Set up an optimizer for updating the parameters of the rnn_clf
rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.001)
    # Number of epochs (passes through the dataset) to train the model for.
num_epochs = 6

#### train model:

In [11]:
train_loss = []
val_loss = []
performance_matrix = None
val_f1 = []
val_p = []
val_r = []
val_acc = []
train_f1 = []
    # A counter for the number of gradient updates
num_iter = 0
model_index = 0
comparable = []
for epoch in range(num_epochs):
    print("Starting epoch {}".format(epoch + 1))
    for (__, example_text, example_lengths, labels) in train_dataloader_mohx:
        example_text = Variable(example_text)
        example_lengths = Variable(example_lengths)
        labels = Variable(labels)
        if using_GPU:
            example_text = example_text.cuda()
            example_lengths = example_lengths.cuda()
            labels = labels.cuda()
            # predicted shape: (batch_size, seq_len, 2)
        predicted = RNNseq_model(example_text, example_lengths)
        batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
        rnn_optimizer.zero_grad()
        batch_loss.backward()
        rnn_optimizer.step()
        num_iter += 1
            # Calculate validation and training set loss and accuracy every 200 gradient updates
        if num_iter % 50 == 0:
            avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_mohx, RNNseq_model,
                                                             loss_criterion, using_GPU)
            val_loss.append(avg_eval_loss)
            val_p.append(performance_matrix[1][0])
            val_r.append(performance_matrix[1][1])
            val_f1.append(performance_matrix[1][2])
            val_acc.append(performance_matrix[1][3])
            print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))

            avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_mohx, RNNseq_model,
                                                              loss_criterion, using_GPU)
            train_loss.append(avg_eval_loss)
            train_f1.append(performance_matrix[1][2])
            print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))
print("Training done for fold {}".format(i))

Starting epoch 1
PRFA performance for  words that are not focus verbs nan nan nan 100.0
PRFA performance for  focus verb nan 0.0 nan 53.3
Iteration 50. Validation Loss 0.3420531749725342.
PRFA performance for  words that are not focus verbs 0.0 nan nan 99.99828578005578
PRFA performance for  focus verb 40.0 0.04698144233027954 0.09385265133740028 52.68888888888889
Iteration 50. Training Loss 0.3445946276187897.
PRFA performance for  words that are not focus verbs nan nan nan 100.0
PRFA performance for  focus verb nan 0.0 nan 53.3
Iteration 100. Validation Loss 0.3192408084869385.
PRFA performance for  words that are not focus verbs 0.0 nan nan 99.99951022287308
PRFA performance for  focus verb nan 0.0 nan 52.7
Iteration 100. Training Loss 0.3218229115009308.
PRFA performance for  words that are not focus verbs nan nan nan 100.0
PRFA performance for  focus verb nan 0.0 nan 53.3
Iteration 150. Validation Loss 0.3001689016819.
PRFA performance for  words that are not focus verbs 0.0 nan n

KeyboardInterrupt: 

### График F1 и Loss:

In [1]:
#plt.figure(0)
plt.figure(figsize=(20, 10))
#plt.title('F1 for moh-x dataset on fold ' + str(i))
plt.title('F1', fontsize='xx-large')
plt.xlabel('iteration', fontsize='xx-large')
plt.ylabel('F1', fontsize='xx-large')
plt.plot(val_f1[:20], 'g')
plt.plot(train_f1[:20], 'b')
plt.legend(['Validation F1', 'Training F1'], loc='best', fontsize='xx-large')
plt.grid()
plt.show()

NameError: name 'plt' is not defined

In [2]:
#plt.figure(1)
plt.figure(figsize=(20, 10))
#plt.title('Loss for moh-x dataset on fold ' + str(i))
plt.title('Loss', fontsize='xx-large')
plt.xlabel('iteration', fontsize='xx-large')
plt.ylabel('Loss', fontsize='xx-large')
plt.plot(val_loss, 'g')
plt.plot(train_loss, 'b')
plt.legend(['Validation loss', 'Training loss'], loc='best', fontsize='xx-large')
plt.grid()
plt.show()

NameError: name 'plt' is not defined

Лучшие f1:

In [3]:
print('val_f1: ', val_f1)
idx = 0
if math.isnan(max(val_f1)):
    optimal_f1s.append(max(val_f1[6:]))
    idx = val_f1.index(optimal_f1s[-1])
    optimal_ps.append(val_p[idx])
    optimal_rs.append(val_r[idx])
    optimal_accs.append(val_acc[idx])
else:
    optimal_f1s.append(max(val_f1))
    idx = val_f1.index(optimal_f1s[-1])
    optimal_ps.append(val_p[idx])
    optimal_rs.append(val_r[idx])
    optimal_accs.append(val_acc[idx])

NameError: name 'val_f1' is not defined

## Результаты работы:

In [4]:
#print('F1 on MOH-X by 10-fold = ', optimal_f1s)
print('Precision  = ', np.mean(np.array(optimal_ps)))
print('Recall  = ', np.mean(np.array(optimal_rs)))
print('F1  = ', np.mean(np.array(optimal_f1s)))
print('Accuracy  = ', np.mean(np.array(optimal_accs)))

NameError: name 'np' is not defined